In [5]:
import random

In [6]:
def distance(node1, node2):
    x = (nodes[node1][1] - nodes[node2][1])**2
    y = (nodes[node1][2] - nodes[node2][2])**2
    return (x+y)**(1/2)

In [7]:
def communication():
    for i in range(len(info_clustering)):
        communication_list = []
        for node1 in info_clustering[i]:
            for agent in on_nodes[node1]:
                communication_list.append(agent)
        for index_1 in range(len(communication_list)):
            for index_2 in range(index_1+1, len(communication_list)):
                agent_1 = communication_list[index_1]
                agent_2 = communication_list[index_2]
                if agent_1 not in info_speed[agent_2]:
                    info_speed[agent_1].append(agent_2)
                    info_speed[agent_2].append(agent_1)
                for agent in info_speed[agent_1]:
                    if agent not in info_speed[agent_2]:
                        info_speed[agent_2].append(agent)
                for agent in info_speed[agent_2]:
                    if agent not in info_speed[agent_1]:
                        info_speed[agent_1].append(agent)
                for j in range(num_nodes):
                    for k in range(num_nodes):
                        s = state_map[agent_1][j][k]+state_map[agent_2][j][k]
                        state_map[agent_1][j][k] = s
                        state_map[agent_2][j][k] = s

In [8]:
def clustering():
    check_list = []
    count = -1
    clustering_info = []
    for index in range(num_nodes):
        prev_check_list = []
        if index not in check_list:
            count+=1
            clustering_info.append([])
            check_list.append(index)
            prev_check_list.append(index)
            clustering_info[count].append(index)
            while(len(prev_check_list)!=0):
                next_check_list = []
                for j in range (len(prev_check_list)):
                    node1 = prev_check_list[j]
                    i = 0
                    while(i<len(nodes_dis[node1]) and nodes_dis[node1][i][1]<=t_constraint):
                        node2 = nodes_dis[node1][i][0]
                        if node2 not in check_list:
                            check_list.append(node2)
                            next_check_list.append(node2)
                            clustering_info[count].append(node2)
                        i+=1    
                prev_check_list = next_check_list
    return clustering_info

In [9]:
num_nodes = 5
print(num_nodes)
num_edges = random.randint((num_nodes-1)*(num_nodes-2)/2+1,num_nodes*(num_nodes-1)/2)
print(num_edges)

nodes = []
state = []
edge_len = [[0]*num_nodes for i in range(num_nodes)]
god_map = [[0]*num_nodes for i in range(num_nodes)]
nodes_dis = []
on_nodes = []
info_speed = []

random_list = [0] * num_nodes

for i in range(num_nodes):
    random_list[i] = i
    state.append([])
    nodes_dis.append([])
    on_nodes.append([])
    
for i in range(num_nodes):
    nodes.append([i,random.randint(0,1000),random.randint(0,1000)])
    for j in range(i-1, -1, -1):
        dis = distance(i, j)
        nodes_dis[i].append((j, dis))
        nodes_dis[j].append((i, dis))

for i in range(len(nodes_dis)):
    nodes_dis[i].sort(key=lambda nodes_dis: nodes_dis[1])
print(nodes_dis)
    
for i in range(num_edges):
    flag_input = 0
    while flag_input == 0: 
        sample_list = random.sample(random_list,2)
        length = random.randint(0,1000)
        a = sample_list[0]
        b = sample_list[1]
        if b not in state[a]:
            state[a].append(b)
            state[b].append(a)
            edge_len[a][b] = length
            edge_len[b][a] = length
            flag_input = 1
        else:
            flag_input = 0
#print(state)
k_agents = int(input("k: "))
t_constraint = int(input("t_constraint: "))
now_point = [0] * k_agents
speed = [0]*k_agents
target = [0]*k_agents
location = [0]*k_agents
x_agent = []
y_agent = []
for i in range(k_agents):
    x_agent.append([])
    y_agent.append([])
    info_speed.append([])
    now_point[i] = random.randint(0,num_nodes-1)
    speed[i] = random.randint(1,10)
print(now_point)
print(state)
print(speed)
print(edge_len)

5
10
[[(3, 225.53491969094276), (1, 400.12498047485116), (4, 471.00955404322747), (2, 743.6612131878333)], [(3, 322.5926223582926), (0, 400.12498047485116), (2, 427.61197363965385), (4, 519.3168589599225)], [(1, 427.61197363965385), (4, 534.2527491740216), (3, 551.5305975193036), (0, 743.6612131878333)], [(0, 225.53491969094276), (4, 268.0), (1, 322.5926223582926), (2, 551.5305975193036)], [(3, 268.0), (0, 471.00955404322747), (1, 519.3168589599225), (2, 534.2527491740216)]]
k: 3
t_constraint: 500
[1, 3, 3]
[[2, 3, 1, 4], [3, 0, 4, 2], [3, 0, 4, 1], [2, 1, 4, 0], [3, 2, 1, 0]]
[7, 3, 2]
[[0, 586, 823, 535, 465], [586, 0, 784, 875, 451], [823, 784, 0, 285, 530], [535, 875, 285, 0, 127], [465, 451, 530, 127, 0]]


In [10]:
history_route = []
state_map = []
for i in range(k_agents):
    history_route.append([])
    history_route[i].append(now_point[i])
    on_nodes[now_point[i]].append(i)
    state_map.append([]) 
for i in range(k_agents):
    state_map[i] = [[0] * num_nodes for i in range(num_nodes)]

In [11]:
finish_count = 0
pre_step = [0]*k_agents
info_clustering = clustering()
while finish_count != num_edges:
    communication()
    for i in range(k_agents):
        list_x = []
        list_y = []
        if target[i]==0:
            pre_step[i] = now_point[i]
            now_point[i] = random.choice(state[now_point[i]])
            target[i] = edge_len[now_point[i]][pre_step[i]]/speed[i]
            #communication(pre_step[i], i)
        location[i]+=1
        draw_flag = 0
        while location[i]>=target[i]:
            state_map[i][now_point[i]][pre_step[i]] += 1
            state_map[i][pre_step[i]][now_point[i]] += 1
            history_route[i].append(now_point[i])
            if god_map[now_point[i]][pre_step[i]]==0:
                finish_count+=1
                god_map[now_point[i]][pre_step[i]] += 1
                god_map[pre_step[i]][now_point[i]] += 1
            
            if(draw_flag == 0):
                pre_x = ((location[i]-1)/target[i])*nodes[now_point[i]][1] + ((target[i] - (location[i]-1))/target[i])*nodes[pre_step[i]][1]
                pre_y = ((location[i]-1)/target[i])*nodes[now_point[i]][2] + ((target[i] - (location[i]-1))/target[i])*nodes[pre_step[i]][2]
                now_x = nodes[now_point[i]][1] 
                now_y = nodes[now_point[i]][2] 
                list_x.append([pre_x, now_x])
                list_y.append([pre_y, now_y])
            else:
                pre_x = nodes[pre_step[i]][1]
                pre_y = nodes[pre_step[i]][2] 
                now_x = nodes[now_point[i]][1] 
                now_y = nodes[now_point[i]][2] 
                list_x.append([pre_x, now_x])
                list_y.append([pre_y, now_y])
            
            pre_step[i] = now_point[i]
            now_point[i] = random.choice(state[now_point[i]])
            location[i] = location[i]-target[i]
            target[i] = edge_len[now_point[i]][pre_step[i]]/speed[i]
            #communication(pre_step[i], i)
            if i in on_nodes[pre_step[i]]:
                on_nodes[pre_step[i]].remove(i)
            draw_flag = 1
        if (draw_flag == 1):
            pre_x = nodes[pre_step[i]][1] 
            pre_y = nodes[pre_step[i]][2]
            now_x = ((location[i])/target[i])*nodes[now_point[i]][1] + ((target[i] - location[i])/target[i])*nodes[pre_step[i]][1]
            now_y = ((location[i])/target[i])*nodes[now_point[i]][2] + ((target[i] - location[i])/target[i])*nodes[pre_step[i]][2]
            list_x.append([pre_x, now_x])
            list_y.append([pre_y, now_y])
            x_agent[i].append(list_x)
            y_agent[i].append(list_y)
        else:
            pre_x = ((location[i]-1)/target[i])*nodes[now_point[i]][1] + ((target[i] - (location[i]-1))/target[i])*nodes[pre_step[i]][1]
            pre_y = ((location[i]-1)/target[i])*nodes[now_point[i]][2] + ((target[i] - (location[i]-1))/target[i])*nodes[pre_step[i]][2]
            now_x = ((location[i])/target[i])*nodes[now_point[i]][1] + ((target[i] - location[i])/target[i])*nodes[pre_step[i]][1]
            now_y = ((location[i])/target[i])*nodes[now_point[i]][2] + ((target[i] - location[i])/target[i])*nodes[pre_step[i]][2]
            x_agent[i].append([pre_x, now_x])
            y_agent[i].append([pre_y, now_y])
        
        if (location[i]/target[i]) > 0.5:
            #communication(now_point[i], i)
            if i not in on_nodes[now_point[i]]:
                on_nodes[now_point[i]].append(i)
        else:
            #communication(pre_step[i], i)
            if i not in on_nodes[pre_step[i]]:
                on_nodes[pre_step[i]].append(i)

In [12]:
cost = 0
for i in range(k_agents):
    if(len(history_route[i]) > cost):
        cost = len(history_route[i])
print(cost)
print(info_speed)

20
[[1, 2, 0], [2, 1, 0], [1, 2, 0]]


In [13]:
print(info_clustering)

[[0, 3, 1, 4, 2]]


In [1]:
pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp37-none-any.whl size=6083 sha256=8b675e1ee1e297c0e29be49c53bd71418a6442af136662e765da2c4bd71eed9e
  Stored in directory: /Users/kuohuichi/Library/Caches/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge ffmpeg

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.1
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/kuohuichi/opt/anaconda3

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    ffmpeg-4.2                 |       h5c2b479_0        43.3 MB  conda-forge
    gnutls-3.6.5               |    h53004b3_1002         2.0 MB  conda-forge
    lame-3.100                 |    h1de35cc_1001         527 KB  conda-forge
    nettle-3.4.1               |    h3efe00b_1002         1.0 MB  conda-forge
    openh264-1.8.0             |    hd9629dc_1000         1.3 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB

In [11]:
def intermediates(p1, p2, nb_points=8): 
    x_spacing = (p2[0] - p1[0])/(nb_points + 1) 
    y_spacing = (p2[1] - p1[1])/(nb_points + 1) 

    return [[p1[0] + i * x_spacing, p1[1] + i * y_spacing] 
      for i in range(1, nb_points+1)] 
# inter_array = intermediates([1, 2], [10, 6.5], nb_points=8)
# print(inter_array) 

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# plt.grid(False)#顯示網格

# plt.axis("equal")#設定了x、y刻度長度一致#需要放在x、ylim指令前
# plt.xlim((0, 1000))#顯示的x的範圍（不設定則由程式自動設定）
# plt.ylim((0, 1000))#顯示的y的範圍
x = []
y = []
for i in range(num_nodes):
    node1 = i
    for node2 in state[i]:
        x.append([nodes[node1][1], nodes[node2][1]])
        y.append([nodes[node1][2], nodes[node2][2]])
# for j in range(k_agents):
#     x_agent.append([])
#     y_agent.append([])
#     for k in range(0, len(history_route[j])-1):
#         node3 = history_route[j][k]
#         node4 = history_route[j][k+1]
#         inter_array = intermediates([nodes[node3][1], nodes[node3][2]], [nodes[node4][1], nodes[node4][2]], nb_points=8)
#         x_agent[j].append([nodes[node3][1], inter_array[0][0]])
#         y_agent[j].append([nodes[node3][2], inter_array[0][1]])
#         for inter_index in range(7):
#             x_agent[j].append([inter_array[inter_index][0], inter_array[inter_index+1][0]])
#             y_agent[j].append([inter_array[inter_index][1], inter_array[inter_index+1][1]])
#         x_agent[j].append([inter_array[7][0], nodes[node4][1]])
#         y_agent[j].append([inter_array[7][1], nodes[node4][2]])

# for i in range(len(x)):
#     plt.plot(x[i], y[i], color='silver')
#     plt.scatter(x[i], y[i], color='r')
# for i in range(len(x1)):
#     plt.plot(x1[i], y1[i], color='k')
# plt.show()

x_1data = []
y_1data = []
x_2data = []
y_2data = []
x_3data = []
y_3data = []

fig, ax = plt.subplots()
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)
line, = ax.plot(0, 0, color = 'silver')
line_1, = ax.plot(0, 0, color='red')
line_2, = ax.plot(0, 0, color='black')
line_3, = ax.plot(0, 0, color='blue')
ball_1 = plt.Circle((5, -5), 18, fc='red')
ball_2 = plt.Circle((5, -5), 18, fc='black')
ball_3 = plt.Circle((5, -5), 18, fc='blue')
def animation_frame(i):
    
    if i >= 0:
        if(i<len(x_agent[0])):
            islist_flag = 0
            for j in range(len(x_agent[0][i])):
                if isinstance(x_agent[0][i][j], list):
                    x_1data.append(x_agent[0][i][j])
                    y_1data.append(y_agent[0][i][j])
                    islist_flag = 1
                    if j == len(x_agent[0][i])-1:
                        ball_1.center = (x_agent[0][i][j][1], y_agent[0][i][j][1])
            if(islist_flag == 0):
                x_1data.append(x_agent[0][i])
                y_1data.append(y_agent[0][i])
                ball_1.center = (x_agent[0][i][1], y_agent[0][i][1])

            line_1.set_xdata(x_1data)
            line_1.set_ydata(y_1data)
            
            

        if(i<len(x_agent[1])):
            islist_flag = 0
            for j in range(len(x_agent[1][i])):
                if isinstance(x_agent[1][i][j], list):
                    x_2data.append(x_agent[1][i][j])
                    y_2data.append(y_agent[1][i][j])
                    islist_flag = 1
                    if j == len(x_agent[0][i])-1:
                        ball_2.center = (x_agent[1][i][j][1], y_agent[1][i][j][1])
            if(islist_flag == 0):
                x_2data.append(x_agent[1][i])
                y_2data.append(y_agent[1][i])
                ball_2.center = (x_agent[1][i][1], y_agent[1][i][1])
                
            line_2.set_xdata(x_2data)
            line_2.set_ydata(y_2data)
            
        if(i<len(x_agent[2])):
            islist_flag = 0
            for j in range(len(x_agent[2][i])):
                if isinstance(x_agent[2][i][j], list):
                    x_3data.append(x_agent[2][i][j])
                    y_3data.append(y_agent[2][i][j])
                    islist_flag = 1
                    if j == len(x_agent[0][i])-1:
                        ball_3.center = (x_agent[2][i][j][1], y_agent[2][i][j][1])
            if(islist_flag == 0):
                x_3data.append(x_agent[2][i])
                y_3data.append(y_agent[2][i])
                ball_3.center = (x_agent[2][i][1], y_agent[2][i][1])

            line_3.set_xdata(x_3data)
            line_3.set_ydata(y_3data)
    
    return line_1,line_2,line_3,ball_1,ball_2,ball_3,
def init():
    line.set_xdata(x)
    line.set_ydata(y)
    
    line_1.set_xdata(x_agent[0][0][0])
    line_1.set_ydata(y_agent[0][0][0])
    
    line_2.set_xdata(x_agent[1][0][0])
    line_2.set_ydata(y_agent[1][0][0])
    
    line_3.set_xdata(x_agent[2][0][0])
    line_3.set_ydata(y_agent[2][0][0])
    
    ball_1.center = (x_agent[0][0][0], y_agent[0][0][0])
    ax.add_patch(ball_1)
    
    ball_2.center = (x_agent[1][0][0], y_agent[1][0][0])
    ax.add_patch(ball_2)
    
    ball_3.center = (x_agent[2][0][0], y_agent[2][0][0])
    ax.add_patch(ball_3)
    
    return line, line_1,line_2,line_3, ball_1,ball_2,ball_3,
    
animation = FuncAnimation(fig, func=animation_frame, frames=np.arange(-2, len(x_agent[0]), 1), init_func=init, interval=10)
plt.show()
animation.save('basic_animation3.mp4')

<Figure size 640x480 with 1 Axes>

In [13]:
print(len(x_1data),len(x_2data),len(x_3data))

2176 2157 2147


In [ ]:
finish_count = 0
pre_step = [0]*k_agents
while finish_count != num_edges:
    for i in range(k_agents):
        if target[i]==0:
            pre_step[i] = now_point[i]
            now_point[i] = random.choice(state[now_point[i]])
            target[i] = edge_len[now_point[i]][pre_step[i]]/speed[i]
            communication(pre_step[i], i)
        location[i]+=1
        while location[i]>=target[i]:
            state_map[i][now_point[i]][pre_step[i]] += 1
            state_map[i][pre_step[i]][now_point[i]] += 1
            history_route[i].append(now_point[i])
            if god_map[now_point[i]][pre_step[i]]==0:
                finish_count+=1
                god_map[now_point[i]][pre_step[i]] += 1
                god_map[pre_step[i]][now_point[i]] += 1
            pre_step[i] = now_point[i]
            now_point[i] = random.choice(state[now_point[i]])
            location[i] = location[i]-target[i]
            target[i] = edge_len[now_point[i]][pre_step[i]]/speed[i]
            communication(pre_step[i], i)
            if i in on_nodes[pre_step[i]]:
                on_nodes[pre_step[i]].remove(i)
        if (location[i]/target[i]) > 0.5:
            communication(now_point[i], i)
            if i not in on_nodes[now_point[i]]:
                on_nodes[now_point[i]].append(i)
        else:
            communication(pre_step[i], i)
            if i not in on_nodes[pre_step[i]]:
                on_nodes[pre_step[i]].append(i)